In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup

%run "catalog_support.py" 
showHeader('FracFocus<br>Big Picture Stats',use_remote=False)

In [ ]:
# fetch data set
master_df = fh.get_df(os.path.join(hndl.curr_repo_dir,'full_df.parquet'))
master_df = master_df[(master_df.date.dt.year>2010)&\
                      (master_df.date.dt.year<=datetime.datetime.now().year)]
master_df = master_df[master_df.in_std_filtered]

   
## Contents:
**[General Statistics](#gen_stats)**
Number of disclosures per week, geographic spread

**[Chemicals used](#chems)** 
Common patterns and chemicals used; trade secrets

**[Water Use](#water_use)**
Weekly: Median, Total and Max water used

**[Proppant Use](#proppant)**
Weekly: Sand, other proppants

**[Companies represented](#companies)** 
Operators and Suppliers

**[Products reported](#products)** 
Trade names of products used

**[Downloadable Data Sets](#data)**
water and sand use, by disclosure

---
<a id='gen_stats'></a>
# General Stats

The figures below use filtered data (no duplicate disclosures or records) for Jan 1, 2011 to the most recently published and curated data.  Because of publishing delays, the most recent months are under represented.

**In the early data, roughly 2011 - May 2013, the bulk download does not include chemical records.**  However, the PDF files for that period are still served through ["Find_A-Well"](https://fracfocus.org/wells/advanced) and document the reported chemicals.  Some projects have attempted to scrape those chemical records into an organized data set but they come with caveats such as they may contain records that the industry no longer considers valid; or they are incomplete because many of the PDFs are poorly formatted and therefore difficult to scrape.  If you are interested in those scraped data sets, contact Open-FF. 

<a id='num_disc'></a>

In [ ]:
gb = master_df.groupby('DisclosureId',as_index=False)['date'].first()
gb1 = master_df.groupby('DisclosureId',as_index=False)['ingKeyPresent'].sum()
mg = pd.merge(gb,gb1,on='DisclosureId',how='left')
gb2 = mg[mg.ingKeyPresent>0].groupby('date').size()
allwk_sum = gb2.resample("W").sum()
ax = allwk_sum.plot(figsize=(12,5), ylabel='Number of disclosures');
ax.set_title('Orange: Without chemical records; Blue: with chemical records',fontsize=10);
plt.suptitle('Weekly number of disclosures by end date in bulk download data',fontsize=15);

gb3 = mg[mg.ingKeyPresent==0].groupby('date').size()
# alldfv1 = master_df[~master_df.ingKeyPresent].groupby('DisclosureId',as_index=False)[['date','TotalBaseWaterVolume']].first()
# gbv1 = gb3.groupby('date').size()
allwk_sumv1 = gb3.resample("W").sum()
allwk_sumv1.plot(ax=ax);



In [ ]:
display(md('The numbers below include only Jan 1, 2011 to the most recent curated data and removes duplicated disclosures and records'))
display(md(f'### Total number of Disclosures: {len(master_df.DisclosureId.unique()):,}'))
display(md(f'### Total number of Records: {len(master_df):,}'))
display(md('The number of records includes non-chemical records'))

## Where are they?
For more detailed maps of each state, see the ["States and Counties"](Open-FF_States_and_Counties.html) pages.

In [ ]:

gb = master_df[master_df.loc_within_state=='YES'].groupby(['bgStateName',
                                                           'DisclosureId'],as_index=False)['bgCAS'].count()
gb = gb.groupby('bgStateName',as_index=False)['DisclosureId'].count().rename({'bgStateName':'StateName',
                                                                           'DisclosureId':'value'},
                                                                          axis=1)
# gb = pd.read_csv(r"C:\MyDocs\OpenFF\src\testing\tmp\temp.csv")
mapping.create_state_choropleth(gb,plotlog=True,custom_scale= [0,1,2,3,4,5,6],
                        legend_name='Number of FracFocus disclosures',
                        start_zoom=3,fields=['StateName','orig_value'],
                        aliases = ['State: ','Number of FF disclosures: ']
                       )


---
<a id='chems'></a>
# Chemicals used
#### [Chemical Index](Open-FF_Chemicals.html)
Browsable list with all reported chemicals. Includes stats and hazard profiles and a links to detailed chemical reports.

#### [Analysis of Substance Classes](https://open-ff.org/the-substance-classes-of-fracfocus-materials/)

#### [Chemical Structure Classifications](https://open-ff.org/fracfocus-chemical-classification-index/)

## Typical Water : Sand : Additive percentages

In [ ]:
# simple diagram with three components
t = pd.DataFrame({'DisclosureId':['median'],'water':[88],'sand':[11],'additives':[1]})
ax = t.plot(x="DisclosureId", y=["water", "sand", "additives"], kind="bar", rot=0, stacked=True,
           ylim=(0,104),width=.85,figsize=(1,3),xlabel='',ylabel="Percent of fracking fluid",
           title="Typical profile",
           legend=False)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.18))
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off

In [ ]:
gb = master_df[master_df.bgCAS=='7732-18-5'].groupby('DisclosureId',as_index=False)['PercentHFJob'].sum()
display(md(f'#### Median percentage of water (CASRN: 7732-18-5) in fracking fluid: {round(gb.PercentHFJob.median())}%'))
gb = master_df[master_df.bgCAS=='14808-60-7'].groupby('DisclosureId',as_index=False)['PercentHFJob'].sum()
display(md(f'#### Median percentage of sand (CASRN: 14808-60-7) in fracking fluid: {round(gb.PercentHFJob.median())}%'))

In [ ]:
# First how many can we resolve to a chemical industry standard ID? (CAS)
c1 = master_df.bgCAS=='ambiguousID'
c2 = master_df.bgCAS=='proprietary'
c3 = master_df.bgCAS=='conflictingID'
c4 = master_df.bgCAS.str[0].str.isnumeric()
totrec = len(master_df[master_df.ingKeyPresent])
rest = len(master_df[~(c1|c2|c3|c4)])
# rest

### How many records are resolvable to an authoritative identity?

In [ ]:
t = pd.DataFrame({'resolvable to CASRN':[len(master_df[c4])],
                  'proprietary':[len(master_df[c2])],
                  'conflictingID':[len(master_df[c3])],
                  'ambiguousID':[len(master_df[c1])],
                  'non-chemical record':[rest]
                  
                })

ax = t.plot.barh(stacked=True,
            figsize=(16,1.5),
           xlabel='all records');
ax.set_title('Resolving chemical identity to a registered CAS number',fontsize=16)
# ax.set_ylabel('all records',fontsize=14)
ax.set_xlabel('Number of FracFocus records',fontsize=14)
plt.xticks(fontsize=14,rotation = 0);
ax = gca().xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', ncol=1, fontsize=14);
# plt.legend(loc='upper left');

### Of the resolvable records, what are the most common?

In [ ]:
t = master_df[master_df.bgCAS.str[0].isin(['0','1','2','3','4','5','6','7','8','9'])]
gb1 = t.groupby('bgCAS',as_index=False).size()
gb2 = t.groupby('bgCAS',as_index=False)[['epa_pref_name']].first()
mg = pd.merge(gb2,gb1,on='bgCAS',how='left')
mg = mg.sort_values('size',ascending=False)
mg.columns = ['Resolved CASRN','EPA preferred name','Number of records']
mg[:50].reset_index(drop=True)

### Number of chemicals added each year
Starting with 2014 because previous years' chemical data is not included in many disclosures.  

In [ ]:
years = range(2014,master_df.date.dt.year.max()+1)
y_in = []
num = []
added = []
last = 0
for y in years:
    c = master_df.date.dt.year<=y
    y_in.append(y)
    n = len(master_df[c].bgCAS.unique())-4
    num.append(n)
    added.append(n-last)
    last = n
t = pd.DataFrame({'year':y_in,'total number of chemical resolved':num,'added this year':added})
iShow(t,paging=False)

### Trade Secret designations
Most states allow operators to declare some materials as trade secrets or proprietary. Although there are many records that are not resolvable to a specific CAS number, Open-FF only categorizes a record as "proprietary" if the record explicitly uses a term like "proprietary" or "confidential business information" in the identity.

More than 80% of disclosures claim at least one chemical as a trade secret.

In [ ]:
testtitle = 'Trade Secret frequency across FracFocus'
# print(alldf.columns)

master_df['is_proprietary'] = master_df.bgCAS=='proprietary'
gb1 = master_df.groupby('DisclosureId',as_index=False)[['is_proprietary','is_valid_cas']].sum()
gb1.fillna(0,inplace=True)
gb1['perc_proprietary'] = gb1.is_proprietary/gb1.is_valid_cas *100
master_df = master_df.merge(gb1[['DisclosureId','perc_proprietary']],
                    on='DisclosureId',how='left',validate='m:1')
# gb = master_df.groupby('DisclosureId
c_plots.proprietary_bars(gb1,testtitle)

---
<a id='water_use'></a>
# Water use

## Gallons used, recorded as `TotalBaseWaterVolume`

In [ ]:
alldf = master_df.groupby('DisclosureId',as_index=False)[['date','TotalBaseWaterVolume','APINumber','bgStateName',
                                                          'OperatorName','bgSupplier']].first()
gb1 = alldf.groupby('date')['TotalBaseWaterVolume'].median()
allwk_tbwv = gb1.resample("W").median()
ax = allwk_tbwv.plot(figsize=(12,5), ylabel='Median Water Volume Used By Week',style='o');
ax.set_title('Median water use (gallons) per week',fontsize=18);
ax = gca().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))


In [ ]:
# alldf[alldf.TotalBaseWaterVolume>100000000]

In [ ]:
gb3 = alldf.groupby('date')['TotalBaseWaterVolume'].sum()
allwk_tbwv_sum = gb3.resample("W").sum()
ax = allwk_tbwv_sum.plot(figsize=(12,5), ylabel='Total Water Volume Used By Week',style='o');
ax.set_title('Total weekly water use across FracFocus',fontsize=18);
plt.suptitle('Data for most recent months probably relects publication delays.',fontsize=10);
ax = gca().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))



In [ ]:
gb1 = alldf.groupby('date')['TotalBaseWaterVolume'].max()

gb2 = gb1[gb1<100000000]
allwk_tbwv = gb2.resample("W").max()
ax = allwk_tbwv.plot(figsize=(12,5), ylabel='Max Water Volume Used By Week',style='o');
ax.set_title('Single maximum water use (gallons) per week',fontsize=18);
plt.suptitle('All values over 100,000,000 gallons excluded - they are probably typos, but we cannot be sure; see table below.',fontsize=10);
ax = gca().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))


In [ ]:
display(md('### Disclosures over 100 million gallons'))
summary = alldf[alldf.TotalBaseWaterVolume>100000000].groupby('DisclosureId',as_index=False)[['date','TotalBaseWaterVolume','APINumber','bgStateName','OperatorName']].first()
summary = summary.drop('DisclosureId',axis=1)
summary.TotalBaseWaterVolume = summary.TotalBaseWaterVolume.map(lambda x: th.round_sig(x,5))
summary.sort_values('date').reset_index()

---
<a id='proppant'></a>
# Proppants
Most 2011- mid 2013 disclosures do not have chemical records.  Proppants are, therefore, missing in those years in the figures below.

The majority of disclosures report "sand" (CASRN: 14808-60-7) as the primary proppant.  Secondary proppants are shown separately below.



In [ ]:
# gb1 = master_df.groupby('DisclosureId',as_index=False)[['APINumber','date','TotalBaseWaterVolume','bgStateName']].first()
c1 = master_df.bgCAS=='14808-60-7'
c2 = master_df.mass<10000000000
gb3 = master_df[c1&c2].groupby('date')[['mass']].sum()

allwk_sand_sum = gb3.resample("W").sum()
otherprop = ['66402-68-4','1302-93-8','1302-76-7','1344-28-1','1318-16-7','308075-07-2','14464-46-1','1302-74-5']

ax = allwk_sand_sum.plot(figsize=(12,5), ylabel='Total 14808-60-7 Mass (lbs) By Week',style='o');
ax.set_title('Total weekly sand (CASRN: 14808-60-7) use across FracFocus',fontsize=18)
plt.suptitle('All values over 10,000,000,000 pounds excluded - they are probably mistakes, but we cannot be sure; see table below.',fontsize=10);
ax = gca().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'));


In [ ]:
# display(md('### Disclosures with sand records over 10 billion pounds'))
# c1 = master_df.bgCAS=='14808-60-7'
# c2 = master_df.mass>10000000000
# summary = master_df[c1&c2][['date','APINumber','bgStateName','OperatorName','mass']].copy()
# summary.mass = summary.mass.map(lambda x: th.round_sig(x,5))
# summary.sort_values('date')


In [ ]:
gb3 = master_df[master_df.bgCAS.isin(otherprop)].groupby('date')[['mass']].sum()
allwk_other_sum = gb3.resample("W").sum()
ax = allwk_other_sum.plot(figsize=(12,5), ylabel='Total other proppants Mass (lbs) By Week',style='o');
ax.set_title('Total weekly use of "other proppants" across FracFocus',fontsize=18)
ax = gca().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'));

props = master_df[master_df.bgCAS.isin(otherprop)].groupby('bgCAS',as_index=False)['epa_pref_name'].first()
print(f'List of "other proppants" graphed: \n{props}\n')

---
<a id='companies'></a>
# Companies represented

## Operators
Operators are the companies that manage the permits, the wells and submit the disclosures.  There is a single operator for each fracking job.  

There are many Operator companies.  A large fraction of them manage only a handful of fracking jobs/wells.  The dominant operators have thousands of wells. To explore individual Operators, use the [Operator Index](Open-FF_Operator_index.html)

The data field in Open-FF named `bgOperatorName` is a form of the FracFocus field `OperatorName` that allows for more comprehensive searches.

In [ ]:
ogb = master_df.groupby(['bgOperatorName','DisclosureId'],as_index=False).size()
ogbname = master_df.groupby(['bgOperatorName'])['OperatorName'].agg(lambda x: x.value_counts().index[0])
ogbname = ogbname.reset_index()
mg = pd.merge(ogb,ogbname,on='bgOperatorName',how='left')
# # ogb = ogb.groupby('bgOperatorName',as_index=False).size()
# # ogb = ogb.sort_values('size',ascending=False)
# # ogb.plot.barh('size','bgOperatorName')
h = mg.OperatorName.value_counts()
print(f'Total number bgOperatorName: {len(h)},\n number with fewer than 20 disclosures: {len(h[h<20])},\n with fewer than 5: {len(h[h<5])}')

In [ ]:
ax = mg.OperatorName.value_counts()[:15].plot.barh(figsize=(8,5))
ax.set_title(f'Top 15 Operators')
ax.set_xlabel('Number of disclosures in FracFocus');
plt.tight_layout();
# plt.savefig(os.path.join(image_dir,'operators.jpg'),dpi=150)

In [ ]:
gb1 = alldf.groupby(['date','OperatorName'],as_index=False).size()
gb1['year'] = gb1.date.dt.year
gb1['month'] = gb1.date.dt.month
gb2 = gb1.groupby(['year','month','OperatorName'],as_index=False).nunique()
gb3 = gb2.groupby(['year','month'],as_index=False)['OperatorName'].count()
gb3['datestr'] = gb3.year.astype('str') + '/' + gb3.month.astype('str') + '/01'
gb3['pltdate'] = pd.to_datetime(gb3.datestr,format='%Y/%m/%d')
ax = gb3.plot('pltdate','OperatorName',figsize=(12,5), ylabel='Number of Operators',
              xlabel='date',style='o',
             legend=False);
ax.set_title('Number of Operators reporting by month',fontsize=18);
ax = gca().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))

## Suppliers

Suppliers are companies that may supply the products used in a fracking job, they may be the oilfield service company or even a transport company.  Because a supplier is associated with single records within a disclosure, there are often multiple companies named as suppliers on a single dislcosure.   

The data field in Open-FF named `bgSupplier` is a form of the FracFocus field `Supplier` that aggregates variations in spellings and typos to allow for more comprehensive searching.

In [ ]:
sgb = master_df.groupby(['bgSupplier','DisclosureId'],as_index=False).size()
sgbname = master_df.groupby(['bgSupplier'])['Supplier'].agg(lambda x: x.value_counts().index[0])
sgbname = sgbname.reset_index()
mg = pd.merge(sgb,sgbname,on='bgSupplier',how='left')

x = ['MISSING','Listed Above','Operator','Customer','Ingredient Container']
mg = mg[~mg.Supplier.isin(x)]
h = mg.Supplier.value_counts()
print(f'Total number bgSupplier: {len(h)},\n number with fewer than 20 disclosures: {len(h[h<20])},\n with fewer than 5: {len(h[h<5])}')

ax = mg.Supplier.value_counts()[:15].plot.barh(figsize=(8,5))
ax.set_xlabel('Number of disclosures in FracFocus');
ax.set_title('Top 15 Suppliers');
plt.tight_layout();
# plt.savefig(os.path.join(image_dir,'suppliers.jpg'),dpi=150)

In [ ]:
gb1 = alldf.groupby(['date','bgSupplier'],as_index=False).size()
gb1['year'] = gb1.date.dt.year
gb1['month'] = gb1.date.dt.month
gb2 = gb1.groupby(['year','month','bgSupplier'],as_index=False).nunique()
gb3 = gb2.groupby(['year','month'],as_index=False)['bgSupplier'].count()
gb3['datestr'] = gb3.year.astype('str') + '/' + gb3.month.astype('str') + '/01'
gb3['pltdate'] = pd.to_datetime(gb3.datestr,format='%Y/%m/%d')
ax = gb3.plot('pltdate','bgSupplier',figsize=(12,5), ylabel='Number of "Suppliers"',
              xlabel='date',style='o',
             legend=False);
ax.set_title('Number of "Suppliers" reported by month',fontsize=18);
ax = gca().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))

---
<a id='products'></a>
# Fracking "Products" Reported
These are the trade named products as reported on disclosures.  

Some names on the disclosure indicate that the product name has been withheld or is disconnected from a chemical record:
>"Ingredient Container"
>
>"MISSING"
>
>"Other Chemicals(s)"

Note that some product names have more than one spelling or capitalization and therefore will be found on more than one line.  
For example, type "lo-surf" into the Search bar.

On some disclosures, operators concatenated all product names onto one line.  When these multiple product records are longer than 30 characters, they are not included in the table below.

In [ ]:
gb1 = master_df.groupby(['DisclosureId','TradeName'],as_index=False).size()
gb1 = gb1.groupby('TradeName',as_index=False).size().rename({'size':'Number of disclosures'},axis=1)
gb1['tn_multiple'] = gb1.TradeName.str.len() > 30 
iShow(gb1[~gb1.tn_multiple][['TradeName','Number of disclosures']].sort_values('Number of disclosures',ascending=False).reset_index(drop=True),
      index=False)

---
<a id='data'></a>
# Summary Data
Data sets that offer quick access to the Open-FF data for commonly requested fields.  (If you have data you would like to access regularly, let us know and we can add a link to this section.)

| Data Set with link | Description |
| :--: | :-- |
| [Water, Sand and BTEX use](scope/water_sand_btex.zip)|- all locations from 2011 to last major update in FracFocus<br>- Total base **water volume** (in gallons)<br><br> *Masses below are in pounds and are for disclosures for which mass is calculable.*<br>- **sand** (CASRN: 14808-60-7) mass<br>- **benzene** (CASRN: 71-43-2) mass<br>- **toluene** (CASRN: 108-88-3) mass<br>- **ethylbenzene** (CASRN: 100-41-4) mass<br>- **xylene**(CASRN: 1330-20-7) mass<br> - "OperatorName" is field as given in FracFocus.<br>- "bgOperatorName" is a generated field to standardize multiple names for the same company.<br>- "APINumber" is a 14-digit number (as text string) from the FracFocus; early disclosures with only 10-digits are filled out with 'XXXX'<br>- "api10" is a simple 10-digit version of the APINumber (as a text string)|
